In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact
from scipy.interpolate import make_interp_spline

class PID:
    def __init__(self, kp=0.5, ki=0.5, kd=0.1, setpoint=1, dt=0.1):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.setpoint = setpoint
        self.dt = dt
        self.clear()

    def clear(self):
        self.integral = 0.0
        self.pre_err = 0.0
        self.feedback_value = 0.0
        self.yp = 0.0

    def compute(self, feedback_value=None):
        if feedback_value is not None:
            self.feedback_value = feedback_value
        err = self.setpoint - self.feedback_value
        self.integral += err * self.dt
        derivative = (err - self.pre_err) / self.dt
        output = (self.kp * err + self.ki * self.integral + self.kd * derivative)
        self.pre_err = err
        self.feedback_value = output
        return output

    def lowpass_filter(self, x, beta):
        y = beta * x + (1 - beta) * self.yp
        self.yp = y
        return y

def simulate_pid(kp=1.0, ki=0.0, kd=0.0):
    pid = PID(kp, ki, kd, setpoint=1, dt=0.1)
    y_list = [0]
    x = 0
    for _ in range(29):
        x = pid.compute(x)
        x = pid.lowpass_filter(x, 0.5)
        y_list.append(x)

    x_list = range(30)
    x_arr = np.array(x_list)
    x_smooth = np.linspace(x_arr.min(), x_arr.max(), 300)
    y_smooth = make_interp_spline(x_list, y_list)(x_smooth)

    plt.figure(figsize=(8, 5))
    plt.plot(x_smooth, y_smooth, label='Response')
    plt.axhline(y=1, color='r', linestyle='--', label='Setpoint')
    plt.title("PID Response")
    plt.xlabel("Time Step")
    plt.ylabel("Output")
    plt.legend()
    plt.grid(True)
    plt.show()

interact(simulate_pid,
         kp=widgets.FloatSlider(
             value=1.0, min=0, max=2.0, step=0.01, description='Kp',
             style={'description_width': 'initial'},
             readout_format='.2f'
         ),
         ki=widgets.FloatSlider(
             value=10.0, min=0, max=15.0, step=0.1, description='Ki',
             style={'description_width': 'initial'},
             readout_format='.2f'
         ),
         kd=widgets.FloatSlider(
             value=0.001, min=0, max=0.005, step=0.0001, description='Kd',
             style={'description_width': 'initial'},
             readout_format='.4f'
         ));
